In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers 
import time
import provider
# import dataset
import glob
import utils


In [2]:
TRAIN_FILES=glob.glob("F:\点云深度学习代码\Dataset\modelnet40_ply_hdf5_2048\*train*.h5")

In [3]:
class self_attention(keras.Model):
    def __init__(self,out_dim,k_head):
       super(self_attention,self).__init__()
       self.out_dim=out_dim
       self.k=k_head
    def build(self,input_shape):
        self.convq=[layers.Conv2D(self.out_dim,1,1) for i in range(self.k)]
        self.convk=[layers.Conv2D(self.out_dim,1,1) for i in range(self.k)]
        self.convv=[layers.Conv2D(self.out_dim,1,1) for i in range(self.k)]
        self.Dense_out=layers.Dense(self.out_dim,activation="relu")
        self.norm=keras.layers.BatchNormalization()
        self.soft=keras.layers.Softmax()
    def call(self,input_feature,training=True):
        out=[]
        for i in range(self.k):
            q=self.convq[i](input_feature)
            v=self.convv[i](input_feature)
            k=self.convk[i](input_feature)
            q=tf.reduce_mean(q,axis=-2,keepdims=True)
            # q=tf.tile(q,[1,1,tf.shape(v)[2],1])
            k=tf.transpose(k,[0,1,3,2])
            qk=tf.matmul(q,k)/tf.sqrt(tf.cast(self.out_dim,tf.float32))
            out.append(tf.matmul(self.soft(qk),v))
        out=tf.concat(out,axis=-1)
        out=self.norm(self.Dense_out(tf.squeeze(out,axis=-2)),training=training)
        return  out 


In [4]:
def random_sample(xyz):
    num=tf.shape(xyz)[1]
    return xyz[:,0:tf.cast(num/2,tf.int32),:]


In [5]:
class pointcloud_class(keras.Model):
    def __init__(self,class_num=10):
        super(pointcloud_class,self).__init__()
        self.class_num=class_num
    def build(self,inputshape):
        self.n=7
        self.Dense1=keras.layers.Dense(64,activation="relu") 
        self.Dense2=keras.layers.Dense(128,activation="relu") 
        self.norm1=keras.layers.BatchNormalization()
        self.norm2=keras.layers.BatchNormalization()
        self.self_attention=[self_attention(128,1) for i in range(self.n+1)]
        self.Dense=[layers.Dense(128,activation="relu") for _ in range(self.n)]
        self.Dense3=layers.Dense(self.class_num)
        self.soft=keras.layers.Softmax()
    def call(self , points_xyz,training=True):
        Density=utils.kernel_density_estimation(points_xyz,1.0)
        featrue=tf.concat([points_xyz,Density],axis=-1)
        featrue=self.Dense1(featrue)
        featrue=self.norm1(featrue,training=training)
        featrue=self.Dense2(featrue)
        featrue=self.norm2(featrue,training=training)
        old_xyz=points_xyz
        for i in range(self.n):
            new_xyz = random_sample(old_xyz)
            gouped_xyz,grouped_feature,idx=utils.grouping(featrue,16,old_xyz,new_xyz)
            new_xyz=tf.reduce_mean(grouped_xyz,axis=-2)
            local_fature=grouped_feature-tf.reduce_mean(grouped_feature,axis=-2,keepdims=True)
            global_fature=tf.reduce_max(grouped_featur,axis=-2,keepdims=False)
            local_featrue=self.self_attention[i](local_fature,training=training)
            featrue=self.Dense[i](tf.concat([local_featrue,global_fature],axis=-1))
            old_xyz=new_xyz
        grouped_xyz, new_points, idx=utils.grouping_all(featrue,new_xyz)
        out=self.self_attention[self.n](new_points,training=training)
        out=tf.squeeze(out,axis=1)
        out=self.Dense3(out)
        out=self.soft(out)
        return out


AttributeError: module 'tensorflow_core.keras.optimizers.schedules' has no attribute 'CosineDecayRestarts'